In [4]:
import cartopy.crs as ccrs
import cartopy.feature as cartopyFeature
import datetime as dt 
import numpy as np
import xarray as xr 

from matplotlib import pyplot as plt
from PIL import Image

In [5]:
import os

In [119]:
casesdir = "/home/guillermo/DEV/EPS/cases"
cases = [
    "pycpt_GTM_EPS_PRCP/-120W--70E_to_-5S-40N",
    "pycpt_GTM_EPS_NIÑO/-170W--120E_to_-5S-5N",
    "pycpt_GTM_EPS_SST_TNA/-55W--15E_to_5S-25N",
    "pycpt_GTM_EPS_UA/-120W--70E_to_-5S-40N"
]

In [120]:
def plot_skills(casedir: str, skills: dict):
    skill_metrics = ['pearson', 'spearman', 'two_alternative_forced_choice', 'roc_area_below_normal', 'roc_area_above_normal']
    limits = [(-1, 1), (-1, 1), (0, 100), (0,1), (0,1)]
    missing_value_flag = -999
    nskills = len(skills.keys())
    fig, ax = plt.subplots(
        nrows=nskills, 
        ncols=len(skill_metrics), 
        subplot_kw={'projection':ccrs.PlateCarree()}, 
        figsize=(3*len(skill_metrics), 3*nskills)
    )
    if nskills == 1: 
        ax = [ax]
    
    i = 0
    for model, skill in skills.items():
        for j, skill_metric in enumerate(skill_metrics):
            cbtitle = skill_metric.upper()
            if skill_metric.startswith("two_alt"):
                cbtitle = "2AFC"
            n = getattr(skill, skill_metric)\
            .where(getattr(skill, skill_metric) > missing_value_flag).plot(
                ax=ax[i][j], vmin=limits[j][0], vmax=limits[j][1],
                cbar_kwargs = {"label": ""}
            )
            ax[i][j].set_xlim(-93,-87)
            ax[i][j].set_ylim(12,19)
            ax[i][j].coastlines()
            ax[i][j].add_feature(cartopyFeature.BORDERS)
            ax[0][j].set_title(cbtitle)

        ax[i][0].text(-0.07, 0.55, model.upper(), va='bottom', ha='center', rotation='vertical', rotation_mode='anchor', 
                      transform=ax[i][0].transAxes)
        i+=1

    fig.savefig(f"{casedir}/figures/cca_models_skillMatrices.png", bbox_inches='tight')    
    plt.close()

In [121]:
def plot_ccamodes(casedir, ccamodes):
    nmodes = 5
    vmin=-10
    vmax = 10
    missing_value_flag = -999
    nmodels = len(ccamodes.keys())
    modelc=-1
    for model, modesdata in ccamodes.items():
        pxs = modesdata["pxs"]
        pys = modesdata["pys"]
        modelc+=1
        fig = plt.figure(figsize=(6*nmodes, 10), layout="tight")
        fig.suptitle(model, fontsize=24)

        for mode in range(nmodes):
            cancorr = np.correlate(pxs.x_cca_scores[:,mode],pys.y_cca_scores[:,mode])
            gs = fig.add_gridspec(3, nmodes, height_ratios=[4,4,2], width_ratios=[1]*nmodes)
            ax1 = fig.add_subplot(gs[0, mode], projection = ccrs.PlateCarree())
            ax2 = fig.add_subplot(gs[1, mode], projection = ccrs.PlateCarree())
            ax3 = fig.add_subplot(gs[2, mode])
            ts_ax = ax3


            map1_ax = ax1
            map2_ax = ax2

            map1_plot = pxs.x_cca_loadings.isel(Mode=mode)\
                .where(pxs.x_cca_loadings.isel(Mode=mode) > missing_value_flag)\
                .plot(ax=map1_ax, cbar_kwargs={"label": "X CCA MODE"})
            map2_plot = pys.y_cca_loadings.isel(Mode=mode)\
                .where(pys.y_cca_loadings.isel(Mode=mode) > missing_value_flag)\
                .plot(ax=map2_ax, cbar_kwargs={"label": "Y CCA MODE"})            

            map1_ax.set_xticks([])
            map1_ax.set_yticks([])
            map2_ax.set_xticks([])
            map2_ax.set_yticks([])
            map1_ax.set_xlabel(None)
            map2_ax.set_xlabel(None)
            map1_ax.set_ylabel(None)
            map2_ax.set_ylabel(None)#map1_ax.set_xlim(-93,-87)
            #map1_ax.set_ylim(12,19)
            #map2_ax.set_xlim(-93,-87)
            #map2_ax.set_ylim(12,19)
            map1_ax.set_title(None)
            map2_ax.set_title(None)
            map1_ax.add_feature(cartopyFeature.BORDERS)
            map2_ax.add_feature(cartopyFeature.BORDERS)
            map1_ax.coastlines()
            map2_ax.coastlines()
            axtitle = f"M{mode+1} - CanCorr={cancorr[0]:.2f}"
            ax1.set_title(axtitle)

            ts = xr.concat(
                [pxs.x_cca_scores.isel(Mode=mode), pys.y_cca_scores.isel(Mode=mode)], 
                'M'
            ).assign_coords({'M': ['x', 'y']})
            primitive = ts.plot.line(marker='x', ax=ts_ax, markersize=8, hue='M')
            ts_ax.grid(axis = 'x', linestyle = '-.')
            ts_ax.spines['top'].set_visible(False)
            ts_ax.spines['right'].set_visible(False)
            ts_ax.spines['bottom'].set_visible(False)
            ts_ax.legend(handles=primitive, labels = list(ts.coords['M'].values), loc="best")
            ts_ax.set_ylabel("CCA Scores")
        print("Guardando CCA modes de modelo", model)
        fig.savefig(f"{casedir}/figures/cca_modes_{model}.png", bbox_inches='tight')    
        plt.close()

In [ ]:
for case in cases:
    print(f"Generando gráficas de skill para el caso {case}")
    casedir = f"{casesdir}/{case}"
    outputdir = f"{casedir}/output"
    outputs = os.listdir(outputdir)
    skills = {}
    for filename in filter(lambda item: "skillscores_cca.nc" in item, outputs):
        model = filename.split(".")[0]
        print(f"Loading data file {filename}")
        skills[model] = xr.open_dataset(f"{outputdir}/{filename}")
    plot_skills(casedir, skills)
    print(f"Generando gráficas de EOF CCA para el caso {case}")
    ccamodes = {}
    for filename in filter(lambda item: "_spatial_loadings.nc" in item, outputs):
        model = filename.split(".")[0]
        print(f"Loading data file {filename}")
        if model not in ccamodes:
            ccamodes[model] = {}
        ccamodes[model]["pxs" if "_x_" in filename else "pys"] = xr.open_dataset(f"{outputdir}/{filename}")
    
    plot_ccamodes(casedir, ccamodes)
    

Generando gráficas de skill para el caso pycpt_GTM_EPS_NIÑO/-170W--120E_to_-5S-5N
Loading data file CanSIPSIC3.SST_skillscores_cca.nc
Loading data file SPEAR.SST_skillscores_cca.nc
Loading data file CFSv2.SST_skillscores_cca.nc
Loading data file SEAS5.SST_skillscores_cca.nc
Loading data file CCSM4.SST_skillscores_cca.nc
Loading data file GEOSS2S.SST_skillscores_cca.nc
Loading data file GLOSEA6.SST_skillscores_cca.nc
Generando gráficas de EOF CCA para el caso pycpt_GTM_EPS_NIÑO/-170W--120E_to_-5S-5N
Loading data file SPEAR.SST_cca_y_spatial_loadings.nc
Loading data file GEOSS2S.SST_cca_y_spatial_loadings.nc
Loading data file GLOSEA6.SST_cca_y_spatial_loadings.nc
Loading data file CanSIPSIC3.SST_cca_y_spatial_loadings.nc
Loading data file GLOSEA6.SST_cca_x_spatial_loadings.nc
Loading data file CCSM4.SST_cca_y_spatial_loadings.nc
Loading data file CanSIPSIC3.SST_cca_x_spatial_loadings.nc
Loading data file SPEAR.SST_cca_x_spatial_loadings.nc
Loading data file SEAS5.SST_cca_x_spatial_loadin

In [123]:
skills['SEAS5']

<xarray.Dataset>
Dimensions:                        (Y: 180, X: 200)
Coordinates:
  * Y                              (Y) float64 39.88 39.62 ... -4.625 -4.875
  * X                              (X) float64 -119.9 -119.6 ... -70.38 -70.12
Data variables:
    pearson                        (Y, X) float64 0.394 0.4306 ... 0.03993
    spearman                       (Y, X) float64 0.4217 0.4496 ... -0.006087
    two_alternative_forced_choice  (Y, X) float64 65.94 66.3 ... 48.91 48.19
    roc_area_below_normal          (Y, X) float64 0.6929 0.7266 ... 0.4 0.5143
    roc_area_above_normal          (Y, X) float64 0.7571 0.7344 ... 0.3852